In [1]:
try:
  import re
  import json
  from IPython.core.display import HTML
  import pandas as pd
  from base64 import b64decode
  from bs4 import BeautifulSoup as bs
except ModuleNotFoundError as e:
  %pip install pandas
  %pip install beautifulsoup4
  %pip install markdown2

In [23]:
fn = 'neural_nets.ipynb'
with open(fn) as f:
  js = json.load(f)['cells']
  df = pd.DataFrame(js)

In [24]:
def writePNG(txt, filename):
  try:
    with open(filename, 'wb') as f:
      f.write(b64decode(txt))
  except Exception as e:
    return False
  return True

In [25]:

df['html'] = df.apply(getHTML,axis=1)
df.head()

,attachments,cell_type,metadata,source,execution_count,outputs,html
0,{},markdown,{},"[## Title\n, This is isghsbajfdjakf]",NaN,NaN,"[\n, [\n], \n, [ Title\n], \n, \n, [This is is..."
1,NaN,code,{},"[try:\n, import networkx as nx\n, impo...",1.0,[],"[\n, [\n], \n, [\n, [\n, <code class=""language..."
2,NaN,code,{},"[def generateNNv(layers, edge_label_on=True,\n...",2.0,[],"[\n, [\n], \n, [\n, [\n, <code class=""language..."
3,NaN,code,{},"[fig, ax = plt.subplots(figsize=(1, 1))\n, ax....",3.0,"[{'data': {'image/svg+xml': ['<?xml version=""1...","[\n, [\n], \n, [\n, [\n, <code class=""language..."
4,NaN,code,{},"[\n, fig, ax = plt.subplots(figsize=(1, 1))\n,...",4.0,"[{'data': {'image/svg+xml': ['<?xml version=""1...","[\n, [\n], \n, [\n, [\n, <code class=""language..."


In [26]:
with open('doc_temp.html') as f:
  page = bs(f,'html')
with open('../html/nav.html') as f:
  nav = bs(f,'html')
page.head.append('')
page.find('div', id='navbar').append(nav)
page.head.title.string = 'Jacobian'
page.find(class_='title').string='Jacobian'
for h in html:
  if len(h)>1:
    page.find('div', id='dynamic_content').append(h)

FileNotFoundError: [Errno 2] No such file or directory: '../html/nav.html'

In [8]:
with open("output1.html", "w", encoding='utf-8') as file:
    file.write(str(page))

In [9]:
def getHTML(row,
            texCapsul_s = '<p class="katex">\n$$\n',
            texCapsul_e ="\n$$\n</p>\n",
            codeCapsule_s = "\n<div>\n<pre>\n<code class='language-python'>",
            codeCapsule_e = "\n</code>\n</pre>\n</div>\n"):

    htm = bs('\n<div>\n</div>', 'html.parser')
    if row['cell_type'] == 'code':
        text = ''
        text += codeCapsule_s
        for s in row['source']:
            if s.strip() == '':
                continue

            text += s
        text += codeCapsule_e
        htm.append(bs(text))
    if row['cell_type'] == 'markdown':
        headerDict = {'#'*i: [f'\n<h{i}>', f'</h{i}>\n'] for i in range(1, 7)}
        found = False
        eq = [texCapsul_s]
        for s in row['source']:
            if s.startswith('\\begin{equation}') or \
            s.startswith('\\begin{equation*}'):
                found = True
                eq.append(s)
            elif found and (s.startswith('\\end{equation}') or
                    s.startswith('\\end{equation*}')):
                eq.append(s)
                eq.append(texCapsul_e)
                htm.append(bs(''.join(eq)))
                found = False
                eq = [texCapsul_s]
            elif found :
                eq.append(s)
            else:
                x = ''
                if s.strip() == '':
                    continue
                if s.strip()[0] == '#':
                    k = s.count('#')*'#'
                    x = headerDict[k][0] + s.replace(k, '') + headerDict[k][1]
                else:
                    x = f'\n<span>{s}</span>\n'
                htm.append(bs(x))
    return htm

def ipynb2html(inFile,
               outFile='',
               tempDoc='doc_temp.html',
               mrkdnTmpl = '{"cells": [{"cell_type": "markdown", "metadata": {}, "source": []}], "metadata": {"language_info": {"name": "python"}, "orig_nbformat": 4}, "nbformat": 4, "nbformat_minor": 2}',
              texCapsul_s = '<p class="katex">\n$$\n',
              texCapsul_e ="\n$$\n</p>\n",
              codeCapsule_s = "\n<div>\n<pre>\n<code class='language-python'>",
              codeCapsule_e = "\n</code>\n</pre>\n</div>\n",
              navF = '../html/nav.html', 
              cssDir='css', 
              jsDir='js'):
    
  with open(inFile) as f:
    df = pd.DataFrame( json.load(f)['cells'])
    f.close()
df['html'] = df.apply(getHTML,axis=1)
with open(tempDoc) as f:
    page = bs(f,'html')
with open(navF) as f:
    nav = bs(f,'html')
page.head.append('')
page.find('div', id='navbar').append(nav)
page.head.title.string = 'Jacobian'
page.find(class_='title').string='Jacobian'
for h in df.html:
  if len(h)>1:
    page.find('div', id='dynamic_content').append(h)
if outFile == '':
    outFile = '.'.join(inFile.split('.')[:-1]) + '.html'
with open(outFile, "w", encoding='utf-8') as file:
    file.write(str(page))


'\\begin{equation*}\n\nf(x) = x^{2}y+3z\n\n\\end{equation*}'

In [28]:

o = df[df.cell_type == 'code'].outputs.iloc[5]

In [38]:
o[0]['output_type']

'execute_result'

In [ ]:
for oo in o:
    